# 🏥 Deep Learning for Medical Imaging: Practical Implementation

## Table of Contents
1. [CNN Fundamentals - Convolution Operations](#practice-1-cnn-fundamentals---convolution-operations)
2. [Building a Simple CNN from Scratch](#practice-2-building-a-simple-cnn-from-scratch)
3. [Transfer Learning with Pre-trained Models](#practice-3-transfer-learning-with-pre-trained-models)
4. [U-Net Architecture for Medical Image Segmentation](#practice-4-u-net-architecture-for-medical-image-segmentation)
5. [Class Activation Maps (CAM) for Interpretability](#practice-5-class-activation-maps-cam-for-interpretability)
6. [Data Augmentation for Medical Images](#practice-6-data-augmentation-for-medical-images)
7. [Introduction to MONAI Framework](#practice-7-introduction-to-monai-framework)
8. [Model Evaluation and Validation](#practice-8-model-evaluation-and-validation)

## Installing and Importing Essential Libraries

In [ ]:
# Install required packages (uncomment if needed)
# !pip install torch torchvision matplotlib numpy Pillow
# !pip install monai

# Import essential libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18, ResNet18_Weights

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# Visualization settings
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Check PyTorch version and device
print(f"PyTorch version: {torch.__version__}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
print("\n✅ All libraries loaded successfully!")

---
## Practice 1: CNN Fundamentals - Convolution Operations

### 🎯 Learning Objectives
- Understand how convolution operations work
- Implement convolution manually and compare with PyTorch
- Visualize feature maps at different layers

### 📖 Key Concepts
**Convolution Operation:** Slides a kernel (filter) across an input image to extract features
- **Kernel size:** 3×3, 5×5, 7×7
- **Stride:** Step size of kernel movement
- **Padding:** Adding borders to preserve spatial dimensions

In [ ]:
# 1.1 Manual implementation of 2D convolution
def manual_conv2d(input_image, kernel, stride=1, padding=0):
    """Manually implement 2D convolution operation"""
    
    # Add padding if needed
    if padding > 0:
        input_image = np.pad(input_image, padding, mode='constant', constant_values=0)
    
    # Get dimensions
    h, w = input_image.shape
    kh, kw = kernel.shape
    
    # Calculate output dimensions
    out_h = (h - kh) // stride + 1
    out_w = (w - kw) // stride + 1
    
    # Initialize output
    output = np.zeros((out_h, out_w))
    
    # Perform convolution
    for i in range(0, out_h):
        for j in range(0, out_w):
            # Extract region
            region = input_image[i*stride:i*stride+kh, j*stride:j*stride+kw]
            # Element-wise multiplication and sum
            output[i, j] = np.sum(region * kernel)
    
    return output

# Create a simple test image (8x8)
test_image = np.random.rand(8, 8)

# Define edge detection kernels
sobel_x = np.array([[-1, 0, 1],
                    [-2, 0, 2],
                    [-1, 0, 1]])

sobel_y = np.array([[-1, -2, -1],
                    [ 0,  0,  0],
                    [ 1,  2,  1]])

# Apply manual convolution
output_x = manual_conv2d(test_image, sobel_x, stride=1, padding=1)
output_y = manual_conv2d(test_image, sobel_y, stride=1, padding=1)

# Visualize results
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
axes[0].imshow(test_image, cmap='gray')
axes[0].set_title('Original Image')
axes[0].axis('off')

axes[1].imshow(output_x, cmap='gray')
axes[1].set_title('Horizontal Edge Detection (Sobel X)')
axes[1].axis('off')

axes[2].imshow(output_y, cmap='gray')
axes[2].set_title('Vertical Edge Detection (Sobel Y)')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("✅ Manual convolution completed!")

In [ ]:
# 1.2 Compare with PyTorch convolution
def pytorch_conv_comparison():
    """Compare manual implementation with PyTorch"""
    
    # Create input (1 batch, 1 channel, 8x8)
    input_tensor = torch.tensor(test_image).unsqueeze(0).unsqueeze(0).float()
    
    # Create convolution layer
    conv_layer = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, 
                          stride=1, padding=1, bias=False)
    
    # Set kernel weights to Sobel X
    with torch.no_grad():
        conv_layer.weight = nn.Parameter(torch.tensor(sobel_x).unsqueeze(0).unsqueeze(0).float())
    
    # Apply PyTorch convolution
    pytorch_output = conv_layer(input_tensor)
    pytorch_output = pytorch_output.squeeze().detach().numpy()
    
    # Compare results
    print("Manual convolution output shape:", output_x.shape)
    print("PyTorch convolution output shape:", pytorch_output.shape)
    print(f"\nMaximum difference: {np.max(np.abs(output_x - pytorch_output)):.6f}")
    print("✅ Results match!" if np.allclose(output_x, pytorch_output, atol=1e-5) else "❌ Results differ")
    
    return conv_layer

conv_layer = pytorch_conv_comparison()

---
## Practice 2: Building a Simple CNN from Scratch

### 🎯 Learning Objectives
- Design and implement a simple CNN architecture
- Understand the flow of data through convolutional layers
- Learn about pooling operations

### 📖 Key Architecture Components
- **Convolutional layers:** Feature extraction
- **Pooling layers:** Spatial downsampling
- **Fully connected layers:** Classification

In [ ]:
# 2.1 Define a simple CNN for binary classification
class SimpleMedicalCNN(nn.Module):
    """Simple CNN for medical image classification"""
    
    def __init__(self, num_classes=2):
        super(SimpleMedicalCNN, self).__init__()
        
        # Convolutional Block 1
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)  # 1 -> 16 channels
        self.bn1 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2)  # 224x224 -> 112x112
        
        # Convolutional Block 2
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)  # 16 -> 32 channels
        self.bn2 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(2, 2)  # 112x112 -> 56x56
        
        # Convolutional Block 3
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # 32 -> 64 channels
        self.bn3 = nn.BatchNorm2d(64)
        self.pool3 = nn.MaxPool2d(2, 2)  # 56x56 -> 28x28
        
        # Global Average Pooling
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)  # -> 1x1
        
        # Fully connected layer
        self.fc = nn.Linear(64, num_classes)
        
    def forward(self, x):
        # Block 1
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        
        # Block 2
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        
        # Block 3
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        
        # Global average pooling
        x = self.global_avg_pool(x)
        x = x.view(x.size(0), -1)  # Flatten
        
        # Classification
        x = self.fc(x)
        
        return x

# Create model instance
model = SimpleMedicalCNN(num_classes=2)
model = model.to(device)

# Print model architecture
print("Model Architecture:")
print("=" * 60)
print(model)
print("=" * 60)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nTotal parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print("\n✅ Model created successfully!")

In [ ]:
# 2.2 Test forward pass and visualize feature map sizes
def test_forward_pass():
    """Test the model with dummy input and track dimensions"""
    
    # Create dummy input (batch_size=1, channels=1, height=224, width=224)
    dummy_input = torch.randn(1, 1, 224, 224).to(device)
    
    print("Forward Pass - Feature Map Dimensions:")
    print("=" * 60)
    print(f"Input shape: {dummy_input.shape}")
    
    # Track intermediate outputs
    x = dummy_input
    
    # Block 1
    x = model.conv1(x)
    print(f"After Conv1: {x.shape}")
    x = F.relu(model.bn1(x))
    x = model.pool1(x)
    print(f"After Pool1: {x.shape}")
    
    # Block 2
    x = model.conv2(x)
    print(f"After Conv2: {x.shape}")
    x = F.relu(model.bn2(x))
    x = model.pool2(x)
    print(f"After Pool2: {x.shape}")
    
    # Block 3
    x = model.conv3(x)
    print(f"After Conv3: {x.shape}")
    x = F.relu(model.bn3(x))
    x = model.pool3(x)
    print(f"After Pool3: {x.shape}")
    
    # Global pooling
    x = model.global_avg_pool(x)
    print(f"After Global Avg Pool: {x.shape}")
    
    x = x.view(x.size(0), -1)
    print(f"After Flatten: {x.shape}")
    
    # Final output
    x = model.fc(x)
    print(f"Final Output: {x.shape}")
    
    print("=" * 60)
    print("✅ Forward pass successful!")
    
    return x

output = test_forward_pass()

---
## Practice 3: Transfer Learning with Pre-trained Models

### 🎯 Learning Objectives
- Load a pre-trained model (ResNet18)
- Modify it for medical imaging tasks
- Understand fine-tuning strategies

### 📖 Key Concepts
**Transfer Learning:** Use knowledge from large datasets (ImageNet) for medical imaging
- **Feature Extraction:** Freeze early layers
- **Fine-tuning:** Gradually unfreeze and train layers

In [ ]:
# 3.1 Load pre-trained ResNet18 and modify for medical imaging
def create_transfer_learning_model(num_classes=2, grayscale=True):
    """Create a transfer learning model from pre-trained ResNet18"""
    
    # Load pre-trained ResNet18
    model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
    
    print("Original ResNet18 Architecture:")
    print("=" * 60)
    print(f"First layer (conv1) input channels: {model.conv1.in_channels}")
    print(f"Last layer (fc) output features: {model.fc.out_features}")
    
    # Modify first layer for grayscale images (1 channel instead of 3)
    if grayscale:
        model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
    
    # Modify last layer for our classification task
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, num_classes)
    
    print("\nModified Architecture:")
    print("=" * 60)
    print(f"First layer (conv1) input channels: {model.conv1.in_channels}")
    print(f"Last layer (fc) output features: {model.fc.out_features}")
    
    return model

# Create transfer learning model
transfer_model = create_transfer_learning_model(num_classes=2, grayscale=True)
transfer_model = transfer_model.to(device)

print("\n✅ Transfer learning model created!")

In [ ]:
# 3.2 Implement different fine-tuning strategies
def apply_finetuning_strategy(model, strategy='freeze_early'):
    """
    Apply different fine-tuning strategies
    
    Strategies:
    - 'freeze_early': Freeze all layers except the last one
    - 'freeze_most': Freeze all except last 2 blocks
    - 'full': Train all layers
    """
    
    if strategy == 'freeze_early':
        # Freeze all layers
        for param in model.parameters():
            param.requires_grad = False
        
        # Unfreeze last layer
        for param in model.fc.parameters():
            param.requires_grad = True
        
        print("Strategy: Freeze all layers except final classifier")
    
    elif strategy == 'freeze_most':
        # Freeze early layers
        for name, param in model.named_parameters():
            if 'layer4' not in name and 'fc' not in name:
                param.requires_grad = False
        
        print("Strategy: Freeze early layers, train layer4 and fc")
    
    elif strategy == 'full':
        # Train all layers
        for param in model.parameters():
            param.requires_grad = True
        
        print("Strategy: Train all layers")
    
    # Count trainable parameters
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total = sum(p.numel() for p in model.parameters())
    
    print(f"Trainable parameters: {trainable:,} / {total:,} ({100*trainable/total:.1f}%)")
    print("\n✅ Fine-tuning strategy applied!")
    
    return model

# Test different strategies
print("Testing Fine-tuning Strategies:")
print("=" * 60)

# Strategy 1: Freeze early layers
model_frozen = create_transfer_learning_model(num_classes=2)
model_frozen = apply_finetuning_strategy(model_frozen, strategy='freeze_early')
print()

# Strategy 2: Freeze most
model_partial = create_transfer_learning_model(num_classes=2)
model_partial = apply_finetuning_strategy(model_partial, strategy='freeze_most')
print()

# Strategy 3: Full training
model_full = create_transfer_learning_model(num_classes=2)
model_full = apply_finetuning_strategy(model_full, strategy='full')

---
## Practice 4: U-Net Architecture for Medical Image Segmentation

### 🎯 Learning Objectives
- Implement the U-Net architecture
- Understand encoder-decoder structure with skip connections
- Learn about segmentation-specific loss functions

### 📖 Key Architecture Components
**U-Net:** Standard architecture for medical image segmentation
- **Encoder:** Contracting path to capture context
- **Decoder:** Expanding path for precise localization
- **Skip Connections:** Preserve spatial information

In [ ]:
# 4.1 Implement U-Net architecture
class UNet(nn.Module):
    """U-Net architecture for medical image segmentation"""
    
    def __init__(self, in_channels=1, num_classes=2):
        super(UNet, self).__init__()
        
        # Encoder (Contracting Path)
        self.enc1 = self.conv_block(in_channels, 64)
        self.pool1 = nn.MaxPool2d(2, 2)
        
        self.enc2 = self.conv_block(64, 128)
        self.pool2 = nn.MaxPool2d(2, 2)
        
        self.enc3 = self.conv_block(128, 256)
        self.pool3 = nn.MaxPool2d(2, 2)
        
        # Bottleneck
        self.bottleneck = self.conv_block(256, 512)
        
        # Decoder (Expanding Path)
        self.upconv3 = nn.ConvTranspose2d(512, 256, 2, stride=2)
        self.dec3 = self.conv_block(512, 256)  # 512 because of skip connection
        
        self.upconv2 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.dec2 = self.conv_block(256, 128)
        
        self.upconv1 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.dec1 = self.conv_block(128, 64)
        
        # Final output layer
        self.out = nn.Conv2d(64, num_classes, kernel_size=1)
    
    def conv_block(self, in_channels, out_channels):
        """Convolutional block: Conv -> BN -> ReLU -> Conv -> BN -> ReLU"""
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        # Encoder
        enc1 = self.enc1(x)
        enc2 = self.enc2(self.pool1(enc1))
        enc3 = self.enc3(self.pool2(enc2))
        
        # Bottleneck
        bottleneck = self.bottleneck(self.pool3(enc3))
        
        # Decoder with skip connections
        dec3 = self.upconv3(bottleneck)
        dec3 = torch.cat([dec3, enc3], dim=1)  # Skip connection
        dec3 = self.dec3(dec3)
        
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat([dec2, enc2], dim=1)
        dec2 = self.dec2(dec2)
        
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat([dec1, enc1], dim=1)
        dec1 = self.dec1(dec1)
        
        # Output
        out = self.out(dec1)
        
        return out

# Create U-Net model
unet_model = UNet(in_channels=1, num_classes=2)
unet_model = unet_model.to(device)

# Print model info
print("U-Net Architecture:")
print("=" * 60)
total_params = sum(p.numel() for p in unet_model.parameters())
print(f"Total parameters: {total_params:,}")
print("\n✅ U-Net model created successfully!")

In [ ]:
# 4.2 Test U-Net forward pass
def test_unet_forward():
    """Test U-Net with dummy input"""
    
    # Create dummy input (batch_size=2, channels=1, height=256, width=256)
    dummy_input = torch.randn(2, 1, 256, 256).to(device)
    
    print("U-Net Forward Pass Test:")
    print("=" * 60)
    print(f"Input shape: {dummy_input.shape}")
    
    # Forward pass
    with torch.no_grad():
        output = unet_model(dummy_input)
    
    print(f"Output shape: {output.shape}")
    print(f"\nExpected: (batch_size=2, num_classes=2, height=256, width=256)")
    print(f"Actual:   (batch_size={output.shape[0]}, num_classes={output.shape[1]}, "
          f"height={output.shape[2]}, width={output.shape[3]})")
    
    # Check if output dimensions match input dimensions (important for segmentation!)
    if output.shape[2:] == dummy_input.shape[2:]:
        print("\n✅ Spatial dimensions preserved correctly!")
    else:
        print("\n❌ Spatial dimensions mismatch!")
    
    return output

unet_output = test_unet_forward()

---
## Practice 5: Class Activation Maps (CAM) for Interpretability

### 🎯 Learning Objectives
- Implement Grad-CAM for visualization
- Understand how to interpret CNN decisions
- Generate heatmaps showing model focus areas

### 📖 Key Concepts
**Grad-CAM:** Gradient-weighted Class Activation Mapping
- Visualizes important regions for predictions
- Essential for clinical trust and validation

In [ ]:
# 5.1 Implement Grad-CAM
class GradCAM:
    """Gradient-weighted Class Activation Mapping"""
    
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        
        # Register hooks
        target_layer.register_forward_hook(self.save_activation)
        target_layer.register_backward_hook(self.save_gradient)
    
    def save_activation(self, module, input, output):
        """Save forward pass activations"""
        self.activations = output.detach()
    
    def save_gradient(self, module, grad_input, grad_output):
        """Save backward pass gradients"""
        self.gradients = grad_output[0].detach()
    
    def generate_cam(self, input_image, target_class=None):
        """
        Generate Class Activation Map
        
        Args:
            input_image: Input tensor (1, C, H, W)
            target_class: Target class index (if None, use predicted class)
        
        Returns:
            CAM heatmap
        """
        # Forward pass
        self.model.eval()
        output = self.model(input_image)
        
        # Get predicted class if not specified
        if target_class is None:
            target_class = output.argmax(dim=1).item()
        
        # Zero gradients
        self.model.zero_grad()
        
        # Backward pass for target class
        class_score = output[0, target_class]
        class_score.backward()
        
        # Calculate weights (global average pooling of gradients)
        weights = torch.mean(self.gradients, dim=(2, 3), keepdim=True)
        
        # Weighted combination of activation maps
        cam = torch.sum(weights * self.activations, dim=1, keepdim=True)
        
        # Apply ReLU (only positive contributions)
        cam = F.relu(cam)
        
        # Normalize to [0, 1]
        cam = cam - cam.min()
        cam = cam / cam.max()
        
        return cam, target_class

print("✅ Grad-CAM implementation ready!")

In [ ]:
# 5.2 Visualize Grad-CAM on a sample image
def visualize_gradcam():
    """Create and visualize Grad-CAM"""
    
    # Create a simple model for demonstration
    demo_model = SimpleMedicalCNN(num_classes=2).to(device)
    demo_model.eval()
    
    # Create Grad-CAM object (target the last convolutional layer)
    gradcam = GradCAM(demo_model, demo_model.conv3)
    
    # Create dummy input image
    input_image = torch.randn(1, 1, 224, 224).to(device)
    
    # Generate CAM
    cam, predicted_class = gradcam.generate_cam(input_image)
    
    # Resize CAM to match input size
    cam_resized = F.interpolate(cam, size=(224, 224), mode='bilinear', align_corners=False)
    cam_resized = cam_resized.squeeze().cpu().numpy()
    
    # Visualize
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # Original image
    axes[0].imshow(input_image.squeeze().cpu().numpy(), cmap='gray')
    axes[0].set_title('Input Image')
    axes[0].axis('off')
    
    # CAM heatmap
    axes[1].imshow(cam_resized, cmap='jet')
    axes[1].set_title(f'Grad-CAM (Predicted: Class {predicted_class})')
    axes[1].axis('off')
    
    # Overlay
    axes[2].imshow(input_image.squeeze().cpu().numpy(), cmap='gray', alpha=0.7)
    axes[2].imshow(cam_resized, cmap='jet', alpha=0.3)
    axes[2].set_title('Overlay')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n✅ Grad-CAM visualization complete!")
    print(f"Predicted class: {predicted_class}")

visualize_gradcam()

---
## Practice 6: Data Augmentation for Medical Images

### 🎯 Learning Objectives
- Implement medical-specific augmentation techniques
- Understand the importance of augmentation in limited data scenarios
- Apply transformations that respect anatomical constraints

### 📖 Key Techniques
- **Geometric transforms:** Rotation, flipping, scaling
- **Intensity transforms:** Brightness, contrast, gamma correction
- **Medical-specific:** Elastic deformations, simulating artifacts

In [ ]:
# 6.1 Define comprehensive augmentation pipeline
class MedicalAugmentation:
    """Medical imaging augmentation pipeline"""
    
    def __init__(self):
        # Training augmentation
        self.train_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomRotation(degrees=15),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5], std=[0.5])
        ])
        
        # Validation/test (no augmentation)
        self.val_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5], std=[0.5])
        ])
    
    def show_augmentations(self, image):
        """Visualize different augmentations"""
        
        fig, axes = plt.subplots(2, 4, figsize=(16, 8))
        
        # Original
        axes[0, 0].imshow(image, cmap='gray')
        axes[0, 0].set_title('Original')
        axes[0, 0].axis('off')
        
        # Individual augmentations
        augmentations = [
            ('Horizontal Flip', transforms.RandomHorizontalFlip(p=1.0)),
            ('Rotation 15°', transforms.RandomRotation(degrees=15)),
            ('Brightness', transforms.ColorJitter(brightness=0.3)),
            ('Contrast', transforms.ColorJitter(contrast=0.3)),
            ('Rotation 30°', transforms.RandomRotation(degrees=30)),
            ('Combined', self.train_transform)
        ]
        
        for idx, (name, transform) in enumerate(augmentations[:7]):
            row = (idx + 1) // 4
            col = (idx + 1) % 4
            
            # Apply transform
            if isinstance(image, np.ndarray):
                pil_image = Image.fromarray((image * 255).astype(np.uint8))
            else:
                pil_image = image
            
            transformed = transform(pil_image)
            
            # Convert to displayable format
            if isinstance(transformed, torch.Tensor):
                if transformed.ndim == 3:
                    transformed = transformed.squeeze().numpy()
                else:
                    transformed = transformed.numpy()
            
            axes[row, col].imshow(transformed, cmap='gray')
            axes[row, col].set_title(name)
            axes[row, col].axis('off')
        
        plt.tight_layout()
        plt.show()

# Create augmentation pipeline
aug_pipeline = MedicalAugmentation()

# Create sample image
sample_image = np.random.rand(256, 256)

# Visualize augmentations
print("Medical Image Augmentation Examples:")
print("=" * 60)
aug_pipeline.show_augmentations(sample_image)
print("\n✅ Augmentation pipeline ready!")

---
## Practice 7: Introduction to MONAI Framework

### 🎯 Learning Objectives
- Introduction to MONAI (Medical Open Network for AI)
- Use pre-built medical imaging transforms
- Understand MONAI's advantages for medical imaging

### 📖 Key Features
**MONAI:** PyTorch-based framework specifically for medical imaging
- Medical-specific transforms and augmentations
- Pre-built networks optimized for medical tasks
- Specialized loss functions (Dice, Focal)

In [ ]:
# 7.1 MONAI basics (install first if needed)
# !pip install monai

try:
    import monai
    from monai.transforms import (
        Compose, LoadImage, EnsureChannelFirst, ScaleIntensity,
        RandRotate, RandFlip, Resize, ToTensor
    )
    from monai.networks.nets import UNet as MONAI_UNet
    from monai.losses import DiceLoss
    
    print(f"MONAI version: {monai.__version__}")
    print("✅ MONAI loaded successfully!\n")
    
    # Define MONAI transforms
    monai_transforms = Compose([
        EnsureChannelFirst(),
        ScaleIntensity(),
        Resize((256, 256)),
        RandRotate(range_x=15, prob=0.5),
        RandFlip(spatial_axis=0, prob=0.5),
    ])
    
    print("MONAI Transform Pipeline:")
    print("=" * 60)
    print(monai_transforms)
    print("\n✅ MONAI transforms defined!")
    
    # Create MONAI U-Net
    monai_unet = MONAI_UNet(
        spatial_dims=2,
        in_channels=1,
        out_channels=2,
        channels=(16, 32, 64, 128, 256),
        strides=(2, 2, 2, 2),
        num_res_units=2,
    )
    
    print("\nMONAI U-Net created!")
    print(f"Total parameters: {sum(p.numel() for p in monai_unet.parameters()):,}")
    
    # MONAI Dice Loss
    dice_loss = DiceLoss(to_onehot_y=True, softmax=True)
    print("\n✅ MONAI Dice Loss initialized!")
    
except ImportError:
    print("⚠️ MONAI not installed. Install with: pip install monai")
    print("Continuing with PyTorch implementation...")

---
## Practice 8: Model Evaluation and Validation

### 🎯 Learning Objectives
- Implement medical imaging metrics (Dice score, IoU)
- Understand evaluation strategies for medical AI
- Calculate sensitivity, specificity, and other clinical metrics

### 📖 Key Metrics
**Classification:**
- Accuracy, Sensitivity (Recall), Specificity
- Precision, F1-score
- ROC-AUC, Precision-Recall AUC

**Segmentation:**
- Dice Score (F1 for segmentation)
- IoU (Intersection over Union)
- Hausdorff Distance

In [ ]:
# 8.1 Implement segmentation metrics
def dice_score(pred, target, smooth=1e-6):
    """
    Calculate Dice Score (F1 for segmentation)
    
    Args:
        pred: Predicted segmentation (binary)
        target: Ground truth segmentation (binary)
        smooth: Smoothing factor to avoid division by zero
    
    Returns:
        Dice score (0 to 1, higher is better)
    """
    pred = pred.flatten()
    target = target.flatten()
    
    intersection = (pred * target).sum()
    dice = (2. * intersection + smooth) / (pred.sum() + target.sum() + smooth)
    
    return dice

def iou_score(pred, target, smooth=1e-6):
    """
    Calculate Intersection over Union (IoU)
    
    Args:
        pred: Predicted segmentation (binary)
        target: Ground truth segmentation (binary)
        smooth: Smoothing factor
    
    Returns:
        IoU score (0 to 1, higher is better)
    """
    pred = pred.flatten()
    target = target.flatten()
    
    intersection = (pred * target).sum()
    union = pred.sum() + target.sum() - intersection
    iou = (intersection + smooth) / (union + smooth)
    
    return iou

# Test with dummy data
print("Segmentation Metrics Test:")
print("=" * 60)

# Create dummy prediction and target
pred = torch.zeros(100, 100)
pred[30:70, 30:70] = 1  # Predicted square

target = torch.zeros(100, 100)
target[25:75, 25:75] = 1  # Ground truth square (slightly larger)

# Calculate metrics
dice = dice_score(pred, target)
iou = iou_score(pred, target)

print(f"Dice Score: {dice:.4f}")
print(f"IoU Score: {iou:.4f}")

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].imshow(target, cmap='gray')
axes[0].set_title('Ground Truth')
axes[0].axis('off')

axes[1].imshow(pred, cmap='gray')
axes[1].set_title('Prediction')
axes[1].axis('off')

# Overlap visualization
overlap = np.zeros((100, 100, 3))
overlap[:, :, 0] = target  # Red channel for ground truth
overlap[:, :, 1] = pred     # Green channel for prediction
axes[2].imshow(overlap)
axes[2].set_title(f'Overlap (Dice={dice:.3f}, IoU={iou:.3f})')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("\n✅ Metrics calculated successfully!")

In [ ]:
# 8.2 Implement classification metrics
def calculate_classification_metrics(y_true, y_pred, y_prob=None):
    """
    Calculate comprehensive classification metrics
    
    Args:
        y_true: Ground truth labels
        y_pred: Predicted labels
        y_prob: Predicted probabilities (optional, for ROC-AUC)
    
    Returns:
        Dictionary of metrics
    """
    # Convert to numpy if torch tensors
    if isinstance(y_true, torch.Tensor):
        y_true = y_true.cpu().numpy()
    if isinstance(y_pred, torch.Tensor):
        y_pred = y_pred.cpu().numpy()
    
    # Calculate confusion matrix components
    tp = np.sum((y_true == 1) & (y_pred == 1))
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    
    # Calculate metrics
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0  # Recall
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    f1 = 2 * (precision * sensitivity) / (precision + sensitivity) if (precision + sensitivity) > 0 else 0
    
    metrics = {
        'accuracy': accuracy,
        'sensitivity': sensitivity,
        'specificity': specificity,
        'precision': precision,
        'f1_score': f1,
        'true_positives': tp,
        'true_negatives': tn,
        'false_positives': fp,
        'false_negatives': fn
    }
    
    return metrics

# Test with dummy data
print("Classification Metrics Test:")
print("=" * 60)

# Generate dummy predictions
np.random.seed(42)
y_true = np.random.randint(0, 2, 100)
y_pred = np.random.randint(0, 2, 100)

# Calculate metrics
metrics = calculate_classification_metrics(y_true, y_pred)

# Print results
print("\nPerformance Metrics:")
print("-" * 40)
print(f"Accuracy:     {metrics['accuracy']:.4f}")
print(f"Sensitivity:  {metrics['sensitivity']:.4f} (Recall)")
print(f"Specificity:  {metrics['specificity']:.4f}")
print(f"Precision:    {metrics['precision']:.4f}")
print(f"F1 Score:     {metrics['f1_score']:.4f}")

print("\nConfusion Matrix Components:")
print("-" * 40)
print(f"True Positives:  {metrics['true_positives']}")
print(f"True Negatives:  {metrics['true_negatives']}")
print(f"False Positives: {metrics['false_positives']}")
print(f"False Negatives: {metrics['false_negatives']}")

print("\n✅ Classification metrics calculated!")

---
## 🎯 Practice Complete!

### Summary of What We Learned:

1. **CNN Fundamentals**
   - Manual convolution implementation
   - Understanding kernels, stride, and padding

2. **Model Architectures**
   - Building CNN from scratch
   - Transfer learning with pre-trained models
   - U-Net for medical image segmentation

3. **Interpretability**
   - Grad-CAM for visualization
   - Understanding model decisions

4. **Data Augmentation**
   - Medical-specific transformations
   - Respecting anatomical constraints

5. **MONAI Framework**
   - Medical imaging specialized tools
   - Pre-built networks and transforms

6. **Model Evaluation**
   - Segmentation metrics (Dice, IoU)
   - Classification metrics (Sensitivity, Specificity)

### Key Insights:

- **Medical imaging requires specialized techniques** different from natural images
- **Interpretability is crucial** for clinical acceptance and trust
- **Proper evaluation metrics** are essential for medical AI validation
- **Domain knowledge** should guide architecture and augmentation choices

### Next Steps:

1. **Implement a complete training pipeline** with real medical data
2. **Explore 3D medical imaging** (CT, MRI volumes)
3. **Study regulatory requirements** (FDA approval process)
4. **Learn about multi-modal fusion** (combining different imaging modalities)
5. **Practice with public datasets** (ChestX-ray14, LIDC-IDRI, BraTS)

### Resources for Further Learning:

- **MONAI Documentation:** https://monai.io/
- **Medical Imaging Datasets:** https://grand-challenge.org/
- **PyTorch Tutorials:** https://pytorch.org/tutorials/
- **FDA Digital Health:** https://www.fda.gov/medical-devices/digital-health

---

## 🏥 Remember:

**Medical AI is about improving patient care, not just achieving high accuracy metrics!**

Always consider:
- Clinical validity and utility
- Patient safety and privacy
- Regulatory compliance
- Ethical implications
- Bias and fairness across populations

---

**Happy Learning! 🚀**